In [1]:
from datasets import load_from_disk
import pandas as pd
import re
import ast
import pyperclip
import pprint
import os
from together import Together
from openai import OpenAI
import pandas as pd 
import glob

together_client = Together(api_key=open('/Users/spangher/.togetherai-usc-key.txt').read().strip())
os.environ['OPENAI_API_KEY']=open('/Users/spangher/.openai-isi-project-key.txt').read().strip()
openai_client = OpenAI()

def query_together(prompt, client=together_client):
    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
        messages=[{
                    "role": "system",
                    "content": "You are an experienced journalist."
                },
                {
                    "role": "user",
                    "content": prompt
                }],
        max_tokens=2048,
        temperature=0.1,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>","<|eom_id|>"],
    )
    return response.choices[0].message.content

In [74]:
OUTLINE_FORMAT = '''
The format of your response should be in this sequence:
  1. First, explain your thought process step by step: 
    - What were the central topics of discussion?
    - What specific areas of the subject's background, expertise, or experiences were explored?
    - Were there any recurring themes or questions that seemed to guide the conversation?
    - How did the interviewer probe for deeper insights or follow up on key points?
  2. Now putting this together, in brackets, create an outline that could have served as the interviewer's guide, with 4-6 broad themes or objectives that are directly relevant to the content of the transcript. Do not simply restate parts of the transcript; instead, synthesize the information into coherent, high-level themes that would shape the flow of the interview.
  3. In other words, place the entire outline you generate in brackets: 
    ie. Here is the format of the generated outline: 
        [
            Who is getting interviewered? Please introduce the source being interviewed (name, expert in what field, etc).
            Then, give a brief background summary of the interview topic.

            - Objective/Theme 1:
            - Objective/Theme 2:
            - Objective/Theme 3:
            ...
        ]
'''

def get_outline_prompt(QA_Sequence, use_few_shot=True):
    if use_few_shot:
        few_shot = '''    [Examples]
        Example output 1:
        [
            Source biography: Howard Kurtz is a media critic and host of CNN's "RELIABLE SOURCES," .
            Interview context: The war on terror raises questions about the media's responsibility to report the truth while protecting national security.
    
            - Objective 1: Thoughts on when information serves the public.
                - Follow-up 1: Ways in which criticism overshadows the valuable service journalists play.
            - Objective 2: Thoughts on when information compromises national security.
            - Objective 3: Thoughts the factual vs. emotional impact of information.
            - Objective 5: Other areas or specific events where news coverage was criticized.
        ]
    
        Example output 2: 
        [
            William Schneider is a political analyst.   
            Brief background: President Bush is expected to ask Congress for a $1 billion increase in NASA's funding to support manned missions to the moon.
        
            - Objective 1: Public perception on space missions.
                - Follow-up 1: Differences with the perception in the 1960s.
            - Objective 2: Political motivations
            - Objective 3: Economic/Demographic motivations
            - Objective 4: President's legacy
                - Follow-up 1: Contrast with father's legacy.
        ]
    
        Now it's your turn.''' 
    else:
        few_shot = ''
    
    prompt = f'''
    You are a helpful journalist's assistant. I will give you a transcript of an interview I just conducted.

    Can you summarize my questions to the goals and notes I had going into the interview with? 
    If some questions were clearly asked in follow-up and in response to information provided by the source, please return them separately. 
    Be abstract (do not mention people's names or events) and concise.
    Please return the outline in brackets based on this transcript. 
    Please express it in the following format:

    [
        Source biography: Give a brief biography on the source being interviewed (name, expertise, etc).
        Interview context: Give a brief background summary of the interview topic.
            - Objective 1:
                - Follow-up 1: (if any)
            - Objective 2:
                - Follow-up 1:
                - Follow-up 2:
            - Objective 3:
            ...
    ]

    {few_shot}
    
    Here is a transcript:

    {QA_Sequence}

    Again, be brief, abstract and concise, try to recreate my high-level notes. There are no fixed amount of objectives, 
    but pay attention to which questions are follow-up questions and which are outline-level.
    Write only a few words per outline point.
    '''
    return prompt

In [75]:
interview_examples = [
    """PAULA ZAHN, CNN ANCHOR: In covering the war on terrorism, we in the media often find ourselves performing a delicate balancing act, asking tough questions, and pursuing the public's right to know, while withholding information that could endanger U.S. soldiers or compromise security here at home. Just recently, Pentagon spokeswoman Victoria Clarke said, "The more access there is, including the eyes and ears of the media, the better of we are." But many question if there is too much coverage if the media, or question the kind of coverage that is on the air. Howard Kurtz reports on the media for "The Washington Post." He also is host of CNN's "RELIABLE SOURCES." Welcome back. How are you doing, Howard? 
    HOWARD KURTZ, CNN'S "RELIABLE SOURCES": Great, Paula. 
    ZAHN: So let's talk a little bit more about what Victoria Clark said. We just lifted a small part of an excerpt of her comments. We know that atrocities on both sides were thoroughly reported over the last several weeks, atrocities committed by the Taliban, atrocities committed by the Northern Alliance. Are you seeing anybody out there arguing that that's a bad thing? 
    KURTZ: I certainly think journalists believe that they need to get out there and aggressively report what's happening on both sides, without any kind of white washing, but the truth is, that a majority of the public in wartime believes -- has very little use for freedom of the press. Back during the Gulf War, for example, 60 percent in 
    ZAHN: So, Howard, what kind of reaction do you think there was to that extremely graphic image of a Northern alliance soldier executing what was believed to be a Taliban soldier? You might remember the picture, fully surrounding him with -- given the poll statistics, was the general feeling by the American public was that you shouldn't print a picture like that, because those are the guys we back in the fight against terrorism? 
    KURTZ: Well, there is always the ongoing debate about how graphic should we be in time of war. Do we need to see the pictures of really heart-wrenching brutality. But you know, this notion that we shouldn't report these stories or show these images because the Northern Alliance are our guys, because they're part of the U.S. alliance, I don't have much sympathy for. I mean, it is not our function to simply be stenographers for the Pentagon and official United States point of view. War is a dirty, messy business, and also a dangerous business, as we saw in killing of the four journalists in Afghanistan yesterday. We need to get out -- and we don't have much access, despite of what Victoria Clark says, to U.S. troops. A lot journalists are out there on their own or travelling with the Northern Alliance to try to show the often seamy, undecided war. 
    ZAHN: If you would, shed some light on some of the more specific criticism we hear by the American public of coverage. "the New York Times," for example, has been accused of providing -- quote -- "a blueprint and an invitation to terrorists for an op-ed piece that basically slammed the state of U.S. meat packing plants, and "The Hartford Current," out of Hartford, Connecticut received over 550 complaints from readers who objected to a photograph that was printed showing a baby that had allegedly been killed by the U.S. bombing campaign. Your reaction to those very specific criticisms? Start first with "The New York Times." 
    KURTZ: Well, a number of papers have printed stories about our nuclear plants, or meat-packing plants or water facilities, or sports stadiums adequately protected, and I think those are legitimate journalistic questions to ask. I think some of the stories have been too specific with maps and details, almost like giving an idea to those who might be inclined to mount some kind of terrorist attack. And as far as civilian casualties, you know, I think we need to remind people, as CNN often does, that there were also a lot of civilian on September 11th, but I don't think we ought to airbrush the casualties that are taking place, and those are horrible pictures to look at, of kids, innocent Afghan kids, in some case, dying in war. I think another important criticism is the undercurrent of negativity that has gone through a lot of the war coverage. This is a very embarrassing time for journalists, let's be perfectly blunt about it. For weeks, we had columnist, commentators, analyzers of all stripes, and some straight news reporters saying Afghanistan was another Vietnam, the White House was blowing it, this was going to drag on forever. And guess what, the Taliban collapsed pretty quickly. I think that some of those journalists now stand guilty of premature prognostication. 
    ZAHN: I think we need to make the distinction, those journalists are, particularly as you pointed out in the piece you did yesterday, are people who are paid to have an opinion. They are Not folks like you and me that are supposed to be here to going straight down the middle of the road. 
    KURTZ: Not entirely. There was a front-page story in "USA Today" that said military experts have concluded we are facing a quagmire. There was a news analysis by Johnny Apple, a "New York Times" reporter, using the 'q' word, quagmire. So it wasn't only the columnists, and look, nobody knew what was going to happen in this war. I was surprise that the Taliban was routed as dramatically as they were, but the point is, that's why those kinds of stories can be dangerous, because there are just three, four, five weeks in a war. 
    ZAHN: One of the points that Bob Steel made, who is the director of ethics program at the Poynter Institute, and this is along the lines you were discussion earlier about this delicate balancing act that journalists are watching. "There is an ethical obligation to make sure you are withholding certain information that could, if not revealed or released to authorities, cause great harm to others." Do you agree? 
    KURTZ: I do agree. Journalists are united in saying we should not report anything that would jeopardize the safety of American troops, but at the same time holding back information about raids that did not go well, weapons that did not go well, that is also important information to have, even if it makes unpopular as the bearers sometimes of bad news. 
    ZAHN: And how are were we on the anthrax meter these days, Howard? The last time you came on, you beat us all up about putting too much anthrax coverage on the air. Now that the story has died down, do you think the portion of the stories is more... 
    KURTZ: Yes, I think the anthrax coverage right now is pretty good. It's a little less alarmist than it had been. Obviously, there have been fewer attacks. I think we're all, government officials, experts, journalists, starting to be able to process that difficult anthrax story. There was a time to me it seemed the media, inadvertently, and I don't accuse you of this, Paula, were contributing to scaring people. I think that period has now passed. 
    ZAHN: Well, as you and I both know, there can be a pack mentality when these stories are covered. Thank you, Howard. Howard Kurtz, Have a good holiday if I don't see you before Thursday. 
    KURTZ: Thank you. TO ORDER A VIDEO OF THIS TRANSCRIPT, PLEASE CALL 800-CNN-NEWS OR USE OUR SECURE ONLINE ORDER FORM LOCATED AT www.fdch.com
    """,
    """
    HOLMES: Fifteen minutes past the hour now on this AMERICAN MORNING. As we have been reporting, states around this country are flat-out broke. Florida is doing worse than many, $3.6 billion short. The new governor is in place now and he has promised to make government leaner. And he has done that by introducing his plan, his budget, that would cut corporate and property taxes -- also cut spending. The governor of Florida joins us now, Governor Rick Scott from Tallahassee. Sir, thank you so much for being here. You got a lot of Tea Party support during the campaign. And you presented this budget proposal in front of a group of Tea Party supporters. Is it fair of me to call this a Tea Party budget? 
    GOV. RICK SCOTT (R), FLORIDA: Well, first off, it is cold in the rest of the country. I hope everybody comes and visits Florida. So, we get a lot of new tourists. No, we -- one thing I'm doing is traveling the state, making sure everybody knows what I'm doing. Not just sitting in the state capitol. So, yesterday, I announced my budget plan, is going to get -- it's a jobs budget. We're going to get our state back to work. That was my whole campaign, seven steps to 700,000 jobs over seven years. And so, I gave -- I gave my talk to a group, my budget announcement to a group of Tea Party activists, over 1,000 of them in Eustis, Florida, and it was a great event. And I came back and I did the same in the capitol. But this is -- this is a budget that we're going to watch how we spend money. I'm reducing the size of the state budget by $5 billion. I'm cutting taxes by $2 billion. 
    HOLMES: Well, Governor -- 
    SCOTT: We're going to get our state back to work. 
    HOLMES: -- you called this a jobs budget. It's a jobs budget -- 
    SCOTT: Right. 
    HOLMES: -- that cuts about 8,700 jobs out of the state government. Why was that a necessary step? That's the part that's got a lot of people's attention. You were getting rid of a lot of state workers, a lot of employees, quite frankly, in a state that already has 12 percent unemployment. 
    SCOTT: Sure. Sure. As you know, a lot of those were vacant positions. But what we've got to do is get the money back into the private sector, give it to you, the taxpayer, you as a businessperson, because you'll go and build private sector jobs. My focus is on building private sector jobs. We're going to make sure our taxes are low enough that, you know, everybody wants to be in business says we've got do business in Florida, because this is a state that they treat businesspeople better -- lower taxes, less regulation. We need regulation, but only for things we have to have. So, my focus is jobs, jobs, jobs. I want to be the jobs governor. And that's what I did yesterday, put a jobs budget out. 
    HOLMES: And, Governor, you said you want to compete with other states. And you want to make your state very attractive to business. Your state already has some of the lower corporate tax rates in the country. Some would argue you're not going to make your state that much more attractive. So, at a time when you're already at 12 percent unemployment, at a time when you're about $4 billion in the hole, why cut that revenue out at a time you clearly need it? 
    SCOTT: Well, because -- business people go where they can get the best return. Florida is going to be the place where you get the best return. We're going to have the lowest taxes. We're going to have a governor that understands that businesspeople don't need a bunch of regulation that's killing jobs. You know, the federal government needs to do the exact same thing. As a country, we've got to reduce our taxes. I'm starting in the state of Florida. But, hopefully, the same thing will happen across the country. 
    HOLMES: And, Governor, on that point of cutting -- at least the corporate taxes in the state, you're hoping you can get businesses to create jobs. But there are plenty out there, including the CBO, that would say that's not necessarily what's going to get businesses to hire. Even if they -- we're talking about the president yesterday with the -- talking to the Chamber, they're sitting on money. A lot of businesses are. But they're not ready to hire because there's just not the demand for their products. And in a state at 12 percent unemployment, your people are hurting. They're not necessarily going to run out and start spending their money to spur businesses so they can, therefore, hire. So, is that really going to work? 
    SCOTT: It's clearly going to work. You know, businesspeople, they want to -- they want to build businesses. They want to add more services, add more jobs. They've got to get a return on their money. So, if -- we've got -- one of the things we're dealing with in our state is, you know, Obamacare is killing jobs. It's killing jobs all across the state. People are worried about. Fortunately, it was declared unconstitutional. I hope the federal government stops all those taxes. But, no, reducing taxes puts money back into the private sector, in your pocket. You'll spend the money in a business person's pocket. They'll go take that money and add jobs. That's what my focus is. This is a jobs budget. We are going to get jobs going in our great state. 
    HOLMES: Well, sir, I have to ask you as well, let you address, because one part that got people's attention as you were cutting 8,700 jobs, also asking state employees to contribute to their pensions, 5 percent. That's first time that's happened in that state -- one of the few states that hasn't done that actually. Why is your office, the budget for your office doubling, your governor's office and your increasing staff? Why was that necessary? 
    SCOTT: Sure. Well, so what we -- first off on the pension plan, I want to make sure we have a viable pension plan. I want to make sure that our state workers know that their pension plan's viable. So, I've asked them to contribute their money to it like they do in most states. Now, the budget in the governor's office was increased because I moved the Department of Emergency Management to our budget. That's the only reason. 
    HOLMES: All right. Well, Governor, we appreciate you being here. Many governors having to get creative; some tough decisions are having to be made. But again, your budget's going to have to be approved, though. It doesn't mean all this is going to -- 
    SCOTT: That's right. 
    HOLMES: -- all your requests are going to get OK'd. But, Governor, thank you so much. Hope to see you down the road. 
    SCOTT: Sure. I'll come down, nice and sunny down here. 
    HOLMES: All right. A good spokesman for your state, as well. That's parts of doing business, as well. Thanks so much, Governor. Kiran? 
    CHETRY: Still to come this hour: an Eagle honored in cowboy country. Not just any Eagle. Michael Vick gets the key to the city of Dallas and it's apparently news to the mayor. Also, coming soon to a classroom near you, perhaps mandatory Arabic. We'll tell you how one Texas school district is taking steps to make learning the language a requirement and what's some parents are saying about it. You're watching AMERICAN MORNING. It's 22 minutes past the hour.
    """,
    """
    "
SCOTT SIMON, host: Juan Ignacio Hernandez Nodar, an American citizen who spent most of his life in Miami, embarked on a dangerous career path in the mid-1990s. His chosen profession landed him in a Cuban prison for 13 years. He endured solitary confinement, death threats and a nervous breakdown. He tried to take his own life. Wasn't until last November that he was set free. Juan Hernandez Nodar is a baseball scout. Mr. Hernandez Nodar's crime: signing ballplayers, including pitcher Livan Hernandez, who's now with the Washington Nationals, and spiriting them out of Cuba for life in the Major Leagues. Mr. Hernandez Nodar was scouting Livan Hernandez's half-brother, the star pitcher Orlando El Duque Hernandez, when he was arrested at a ballgame in Cuba in 1996. He was sentenced to 15 years in prison and was finally released last year. Juan Hernandez Nodar joins us from member station WLRN in Miami. Thanks so much for being with us. 
Mr. JUAN IGNACIO HERNANDEZ NODAR (Baseball Agent): My pleasure, sir. 
SCOTT SIMON, host: First, how are you doing? 
Mr. JUAN IGNACIO HERNANDEZ NODAR (Baseball Agent): Well, I'm doing fine. What can I tell you? I've been here almost six months already since I came back from Cuba, and a lot of things have been changed for me. 
SCOTT SIMON, host: Yeah. Well, could I get you to cast back to that day, August 1996, when you were arrested? 
Mr. JUAN IGNACIO HERNANDEZ NODAR (Baseball Agent): Yes, sir. That day I left Miami was August the 10th of '96. I arrived in Havana around 11:00 that night. I rent a car and I drove to my family in the town named San Nicolas. And from there, the next day we keep traveling to all the way to Santi Espiritu. The youth tournament of baseball was taking place that week. And next day I got arrested at noon. When the game between the United States of America and Venezuela finished, I got arrested. And three days later, I (unintelligible) find out, I was arrested in the charge that I was helping the Cuban defectors that left Cuba on the past month. 
SCOTT SIMON, host: May I ask, I mean now that you're safely out, were you? 
Mr. JUAN IGNACIO HERNANDEZ NODAR (Baseball Agent): Were I? 
SCOTT SIMON, host: Were you guilty of that charge? I mean maybe it's an unjust law, but were you in fact... 
Mr. JUAN IGNACIO HERNANDEZ NODAR (Baseball Agent): Let me tell you. I don't think it's any guilty at all, because all those players - Livan, Vladimir Nunez, Larry Rodriquez, Osvaldo Fernandez, which are the players I took charge of them - they left Cuba with a visa to enter the country to play tournaments. When they were there they decide to stay. There's nothing illegal of that. Because you decide to establish your residence over there, that's completely legal. So I believe they wrong with me. 
SCOTT SIMON, host: Mr. Hernandez Nodar, what was it like in prison for you? 
Mr. JUAN IGNACIO HERNANDEZ NODAR (Baseball Agent): Well, I'll be honest with you. Prison in Cuba is awful because - let me put it to you this way. At the beginning, I was in a cell that the capacity of that cell was 36 persons and there were 82 of us in there. We were sleeping in the bathroom, in the floors. Every place we could find a little spot at night, we laid down over there and sleep. And the food over there was awful. At the beginning I lost 63 pounds in less than four months due to not having to eat that type of thing. And believe me, it was awful. Awful. 
SCOTT SIMON, host: You were put in solitary confinement... 
Mr. JUAN IGNACIO HERNANDEZ NODAR (Baseball Agent): Yeah, I was put in solitary in 2000 when the United States baseball team win the Olympics down in Sydney. I was very happy for that and the guard in the floor that I was grabbed me and put in solitary, because I was happy because the United States of America won the Olympics. And I told him, hey, guy, I'm a U.S. citizen and I'm proud to be a U.S. citizen. I'm happy for my country. In the same way every time Cuba wins, you people get happy, I should get happy. We don't care about that. You don't suppose to cheer because the United States wins. It's something, you know, stupid things that if you analyze, they don't have to be like that. There was a time that the prisoner warden told me, Juan, don't say anything else because you belong to Mr. Fidel Castro. You are his personal prisoner and nobody except him will allow you to do something. 
SCOTT SIMON, host: I gather you're in the D.R., the Dominican Republic now, right? 
Mr. JUAN IGNACIO HERNANDEZ NODAR (Baseball Agent): Yes, sir. Since I come back, I went back to D.R., where I got most of my family. And I opened a baseball training camp over there, which the main thing is grab the young Dominican talents who are from 15 to 16 years old and practice them over there, feed them. We got room and board facility where we keep them over there. We teach them the language of English, we're giving classes, and we practice every day. We try to get them to sign with a major league team. And most of those kids come from the streets and from the valleys and from the country, you know, that they decide not to go to school no more and start playing ball, because they think playing baseball is a future for them and their families. 
SCOTT SIMON, host: I'm assuming that perhaps only a small percentage of people at your academy actually make it into the major leagues. 
Mr. JUAN IGNACIO HERNANDEZ NODAR (Baseball Agent): Right now we got 22 kids. 
SCOTT SIMON, host: Yeah. 
Mr. JUAN IGNACIO HERNANDEZ NODAR (Baseball Agent): And out of those 22 kids, I got three of them that baseball teams are interested on them, which they already given them tryouts and stuff like that. And if everything goes right, they should be signed by the end of the month of June or at the beginning of July. 
SCOTT SIMON, host: Which raises the question: What happens to the 19 kids who aren't signed? Are there lives better cause they were at your academy? 
Mr. JUAN IGNACIO HERNANDEZ NODAR (Baseball Agent): Well, in the period they're in our academy, we try to keep them there, like I said, for a year. If we don't see that the kid's going to make it, we go straight to him and tell him, look, this is not for you, we recommend you to go back to school. And we even help them to go back to school. 
SCOTT SIMON, host: Are you still in a position or have any interest in helping Cuban ballplayers? 
Mr. JUAN IGNACIO HERNANDEZ NODAR (Baseball Agent): All the time, sir. Any Cubans that would like to approach to me, my hands, my heart will be open for them, and I'll be more than glad to help them. 
SCOTT SIMON, host: Well, Juan Ignacio Hernandez Nodar, baseball scout, joining us from WLRN in Miami, thanks so much for being with us. 
Mr. JUAN IGNACIO HERNANDEZ NODAR (Baseball Agent): For me it's been a pleasure, sir."
    """,
    """
AUDIE CORNISH, HOST: From NPR News, this is ALL THINGS CONSIDERED. I'm Audie Cornish. And we begin this hour with $171 billion. That's the size of a second rescue package for debt-troubled Greece being negotiated today by European finance ministers meeting in Brussels. They hope the bailout will, finally, ease the economic crisis that's pushed the group of nations that uses the euro to the breaking point. But there are still outstanding issues to resolve and Greece faces huge obstacles, even with the bailout, before it can return to fiscal health. Joining us now from Brussels is NPR's Eric Westervelt. And, Eric, before the European Union and its partners dole out that $171 billion worth of new aid, I understand they want additional assurances from Greece. What are they looking for? 
ERIC WESTERVELT, BYLINE: Well, Audie, finance ministers meeting here tonight, they want more control over how Greece spends this money. I mean, one option I'm told is to set up a kind of escrow account which would try to assure that Greece prioritizes servicing its debt before it pays for any basic budget items and government services. You know, this would be one way to keep the pressure on Greece after they get the money to make, you know, good on reforms and stick with these really tough and painful austerity plans. In the past, Audie, Greek politicians have complained that that kind of mechanism is unacceptable, that it's an infringement on their fiscal sovereignty. But I think as the crisis has deepened and Greece has failed repeatedly to meet many of the goals of the first bailout, you know, there's now a sense in Greece and the rest of Europe that this the reality and this kind of oversight by the EU and its other creditors may have to be done. 
AUDIE CORNISH, HOST: And then there's the issue of whether the numbers really add up here. I mean, what are the biggest concerns there? 
ERIC WESTERVELT, BYLINE: Well, they really want to get Greece's debt down to a manageable level in the long term, so it can really start to expand and grow the economy and go back to the bond market and borrow on reasonable, sustainable terms. But some number crunching being done by the EU shows that even with this second massive bailout and unprecedented, you know, austerity measures and cost-cutting moves, Greece's debt would still be unsustainably high by 2020. One possibility, I'm told, is to reduce the interest rate Greece is going to pay on its first bailout. Another is to have private bond holders take an even bigger loss on Greek bonds. As part of this bailout, private creditors will trade their old Greek bonds for new ones and take a big hit, almost 70 percent loss, but with some favorable terms thrown in. So, it's thought that getting private bond holders to take an even bigger loss, or haircut, will be one way to try to close that gap I mentioned. But that's going to be a tough sell as bond holders are already taking a huge loss. 
AUDIE CORNISH, HOST: And lastly, Eric, is there any sense or fear that this is too little, too late? I mean, this is the second massive cash infusion for Greece in as many years. 
ERIC WESTERVELT, BYLINE: Yeah. I mean, the big question is, you know, will Greece stick with the austerity measures, you know, in the wake of street protests and even after elections? And then, the bigger issue is, you know, will Greece's chronic lack of growth undermine the whole thing? Do the numbers add up? I mean, all the emphasis, Audie, has been on austerity, on cuts. Will Greece be able to do anything to foster and encourage growth? There's concern the hole is now sort of too deep, that Greece's economy is in such bad shape. I mean, it contracted 7 percent last quarter. Unemployment is at 21 percent. You know, it's going to be very, very hard for Greece to, you know, implement these cost-cutting measures and try to help the economy expand and grow. 
AUDIE CORNISH, HOST: That's NPR's Eric Westervelt in Brussels. Eric, thank you so much. 
ERIC WESTERVELT, BYLINE: Thank you, Audie.
"""
]

In [80]:
for ex in interview_examples:
    prompt = get_outline_prompt(ex)
    r = query_together(prompt)
    print(r)
    print()

[
    Source biography: Media critic and journalist, host of a media analysis show.
    Interview context: Balancing media's right to report with national security concerns in wartime.

    - Objective 1: Reporting atrocities in war
        - Follow-up 1: Graphic images and public reaction
    - Objective 2: Criticism of media coverage
        - Follow-up 1: Specific examples of criticism
        - Follow-up 2: Undercurrent of negativity in coverage
    - Objective 3: Journalistic responsibility
        - Follow-up 1: Withholding information to prevent harm
    - Objective 4: Anthrax coverage
]

[
    Source biography: Governor of a state with significant financial struggles.
    Interview context: State budget proposal and economic recovery.

    - Objective 1: Budget proposal and Tea Party influence
        - Follow-up 1: Addressing criticism of job cuts in state government
    - Objective 2: Job creation through tax cuts
        - Follow-up 1: Effectiveness of tax cuts in stimulatin

In [ ]:
""""Here are the key pieces of information extracted from the interview transcript:

Information item #1: The European Union is negotiating a second rescue package for Greece worth $171 billion to ease the economic crisis.

Information item #2: The EU wants additional assurances from Greece, including more control over how Greece spends the bailout money, possibly through an escrow account to prioritize debt servicing over basic budget items and government services.

Information item #3: The EU is concerned that Greece's debt may still be unsustainably high by 2020, even with the second bailout and austerity measures, and is considering options such as reducing interest rates or having private bond holders take a bigger loss on Greek bonds.

Information item #4: Private bond holders will take a 70% loss on Greek bonds as part of the bailout, but may be asked to take an even bigger loss to close the gap.

Information item #5: There are concerns that the bailout may be too little, too late, and that Greece's chronic lack of growth and high unemployment (21%) may undermine the entire effort.

Information item #6: The EU is worried that Greece may not stick to austerity measures in the face of street protests and elections, and that the country's economy may be too damaged to recover.""""